In [1]:
import json
import os
import requests
from dotenv import load_dotenv
from pathlib import Path
#from web3.auto import w3
from web3 import Web3, HTTPProvider

In [2]:
load_dotenv()

True

In [3]:
# create a web3.py instance w3 by connecting to the local Ethereum node
w3 = Web3(HTTPProvider("http://localhost:8545"))

print(w3.isConnected())

# Initialize a local account object from the private key of a valid Ethereum node address
local_acct = w3.eth.account.from_key(os.getenv("ACCT_PRIVATE_KEY"))

# compile your smart contract first, then copy the whole 'FarmsMarket.json' file
truffleFile = json.load(open('FarmersMarket.json'))
abi = truffleFile['abi']
bytecode = truffleFile['data']['bytecode']['object']

# Initialize a contract object with the smart contract compiled artifacts
contract = w3.eth.contract(bytecode=bytecode, abi=abi)

# build a transaction by invoking the buildTransaction() method from the smart contract constructor function
construct_txn = contract.constructor('FarmsMarket', 'FMD').buildTransaction({
    'from': local_acct.address,
    'nonce': w3.eth.getTransactionCount(local_acct.address),
    'gas': 5000000,
    'gasPrice': w3.toWei('21', 'gwei')})

# sign the deployment transaction with the private key
signed = w3.eth.account.sign_transaction(construct_txn, local_acct.key)

# broadcast the signed transaction to your local network using sendRawTransaction() method and get the transaction hash
tx_hash = w3.eth.sendRawTransaction(signed.rawTransaction)
print(tx_hash.hex())

# collect the Transaction Receipt with contract address when the transaction is mined on the network
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
print("Contract Deployed At:", tx_receipt['contractAddress'])
contract_address = tx_receipt['contractAddress']

# Initialize a contract instance object using the contract address which can be used to invoke contract functions
fm_contract = w3.eth.contract(abi=abi, address=contract_address)

True
0x2ca168194901f32f50f1208a83053c8540e965938e9ce03487b0cf1d1f13df4f
Contract Deployed At: 0x2D1A3946635f840827c7bd1aF23139C6eFD48056


In [4]:
# env variables

headers = {
    "Content-Type": "application/json",
    "pinata_api_key": os.getenv("PINATA_API_KEY"),
    "pinata_secret_api_key": os.getenv("PINATA_SECRET_API_KEY"),
}

def convertDataToJSON(content):
    data = {"pinataOptions": {"cidVersion": 1}, "pinataContent": content}
    return json.dumps(data)

def pinJSONtoIPFS(json):
    r = requests.post(
        "https://api.pinata.cloud/pinning/pinJSONToIPFS",
        data=json,headers=headers
    )
    ipfs_hash = r.json()["IpfsHash"]
    return ipfs_hash


In [5]:
# 1. registerVendor
#
#    struct vendor {
#        string URI;
#        address _address;
#    }
def vendor_info():
    name = input("Enter the Name: ")
    phone = input("Enter a Contact Phone: ")
    
    vendor_object = {
            "name": name,
            "phone": phone,
    }

    return vendor_object

# registerVendor(address vendorAddress, string memory vendorURI) 
# public returns(uint)

def register_vendor(vendor_address:str):

    # use contact_info to create URI using pinata API
    vendor_object = vendor_info()
    data = convertDataToJSON(vendor_object)
    ipfs_link = pinJSONtoIPFS(data)

    # call contract to create new owner
    tx_hash = fm_contract.functions.registerVendor(
        vendor_address, ipfs_link)\
        .transact({"from": w3.eth.accounts[0]})

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [6]:
# 2. registerProduct
#
# registerProduct (string memory _type, uint vendorID, 
#                 string memory URI, uint quantity, uint price) 
#    public returns(uint)

def register_product(product_type:str, 
                     vendorID:int,  
                     quantity:int, 
                     price:int):

    # use product_info to create URI using pinata API
    product_object = {
            "product_type": product_type,
            "vendorID": vendorID,
            "quantity": quantity,
            "price": price,
    }
    data = convertDataToJSON(product_object)
    ipfs_link = pinJSONtoIPFS(data)

    # call contract to create new owner
    tx_hash = fm_contract.functions.registerProduct(
        product_type,
        vendorID,
        ipfs_link,
        quantity,
        price)\
        .transact({"from": w3.eth.accounts[0]})

    receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    return receipt

In [7]:
# 3. updateProduct

In [8]:
# 4. removeProduct

In [9]:
# 5. removeVendor

In [10]:
# 6. makePurchase

In [11]:
# 7. updateProductHistory

In [12]:
# 8. returnPurchase

In [13]:
# view funtions for testing
def getLatestVendor():
    ipfs_hash = fm_contract.functions.getLatestVendor().call()
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

def getVendorByID(vendor_id):
    ipfs_hash = fm_contract.functions.vendors(vendor_id).call()[0]
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    return message.json()

In [18]:
## testing

vendor_count = 0;
product_count = 0;

option = input("Would you like to: 1. add a vendor 2. get a vendor 3. add a product 4. get a product?")

if option == "1":
    vendor_address = "0x8ae3f5d6d85cEE8E7F6e41Bad033fDF0e6239322"
    receipt = register_vendor(vendor_address)
    vendor_count += 1

if option == "2":
    vendor_id = input("Enter a vendor ID to retive the vendor. Hit enter to get the latest.")

    if (vendor_id):
        print(getVendorByID(int(vendor_id)))
    else:
        print(getLatestVendor())

if option == "3":
    product_type = input("Enter the product type: ")
    vendorID = input("Enter the vendor ID: ")
    quantity = input("Enter the pruduct quantity: ")
    price = input("Enter the pruduct price:")
  
    receipt = register_product(product_type, 
                               int(vendorID), 
                               int(quantity), 
                               int(price))
    product_count += 1
    
if option == "4":
    product_id = input("Enter a product ID to retive the product. Hit enter to get the latest.")
    if (product_id):
        ipfs_hash = fm_contract.functions.products(int(product_id)).call()[2]
    else:
        ipfs_hash = fm_contract.functions.products(1).call()[2]
    message = requests.get(f"https://cloudflare-ipfs.com/ipfs/{ipfs_hash}")
    print(message.json())   

Would you like to: 1. add a vendor 2. get a vendor 3. add a product 4. get a product? 4
Enter a product ID to retive the product. Hit enter to get the latest. 


{'product_type': 'apple', 'vendorID': 1, 'quantity': 100, 'price': 5}
